In [1]:
from pulp import *
import numpy as np
import pandas as pd

# Постачання
supply = [200, 250, 200]
suppliers = range(len(supply))

# Попит
demand = [190,100,120,110,130]
consumers = range(len(demand))

# Вартість перевезення
costs = [[28, 27,18,27,	24], [18,26,27,32,21], [27,33,23,31,34]]

# Дані для задачі
costs = makeDict([suppliers, consumers], costs, 0)
prob = LpProblem("Transportation_Problem", LpMinimize)

# Шляхи перевезення
routes = [(i, j) for i in suppliers for j in consumers]

# Змінні для шляхів перевезення
vars = LpVariable.dicts("Route", (suppliers, consumers), 0, None, LpInteger)

# Функція мінімізації
prob += (
    lpSum([vars[w][b] * costs[w][b] for (w, b) in routes]),
    "Sum_of_Transporting_Costs",
)
# Максимальні обмеження постачання
for s in suppliers:
    prob += (
        lpSum([vars[s][b] for b in consumers]) <= supply[s],
        "Sum_of_Products_out_of_supplier_%s" % s,
    )

# Мінімальні обмеження попиту
for c in consumers:
    prob += (
        lpSum([vars[w][c] for w in suppliers]) >= demand[c],
        "Sum_of_Products_into_consumer%s" % c,
    )
# Вирішення задачі за допомогою PuLP
prob.solve()

results = [v.varValue for v in prob.variables()]

print("Відповідь:")
print(pd.DataFrame(np.array(results).reshape(3, 5)))
print("Загальна вартість:", value(prob.objective))

Відповідь:
     0    1   2    3   4
0    0  100  30    0  70
1  190    0   0    0  60
2    0    0  90  110   0
Загальна вартість: 15080
